In [ ]:
import numpy as np
import csv
import GPy
from matplotlib import pyplot as plt
import pandas as pd
import warnings
warnings.simplefilter("error", RuntimeWarning)

In [ ]:
def cnt_rmse(y, y_hat):
    return np.sqrt(((y_hat.flatten()-y.flatten())**2).mean())

In [ ]:
fname = '/home/chun/Desktop/gpr_testing/msd/YearPredictionMSD.txt'


df = pd.read_csv(fname, header=None)

df.head(5)

In [ ]:
X = df.iloc[:,1:].values
train_X = X[:463715]
test_X = X[463715:]

Y = np.reshape(df.iloc[:,0].values, (len(X),1))
train_Y = Y[:463715]
test_Y = Y[463715:]

print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

In [ ]:
kernel = GPy.kern.RBF(input_dim = 90, variance = 1., lengthscale = 1., ARD = True)
# kernel = GPy.kern.RBF(input_dim = 90, variance = 1., lengthscale = 1.)
optimize = True
# optimize = False

# train_size = 0
train_size = 5000
# train_size = 4000
# train_size = 3000
# train_size = 2000
# train_size = 1000
# train_size = 500
# train_size = 300
# train_size = 100

data_idx = np.arange(len(train_X))

err_list = []
for _ in range(10):

    # sample new_X
    if train_size == 0:
        new_train_idx = data_idx
    else:
        np.random.shuffle(data_idx)
        new_train_idx = data_idx[:train_size]

    new_X = X[new_train_idx]
#     print(test_X.shape)
    print(new_X.shape)

    # normalize y 
    Y_mean = Y - Y[new_train_idx].mean()
    test_y = Y_mean[463715:]
    new_y = Y_mean[new_train_idx]
#     print(test_y.shape)
#     print(new_y.shape)

    try:
    #         m = GPy.models.GPRegression(new_X, new_y, kernel, normalizer=True)
        m = GPy.models.GPRegression(new_X, new_y, kernel, normalizer=False)
    except RuntimeWarning as e:
        print(e, '0')
    #         continue
    if optimize:
        try:
            m.optimize(max_iters=100, optimizer="lbfgs")
    #         m.optimize(max_iters=100)
        except RuntimeWarning as e:
            print(e, '1')
    #             continue
    try:
        y_hat = m.predict(test_X)[0]
    except RuntimeWarning as e:
        print(e, '2')
    #         continue
    try:
        err = cnt_rmse(y_hat, test_y)
    except RuntimeWarning as e:
        print(e, '3')
    #         continue

    err_list.append(err)

print(sum(err_list)/len(err_list))
